# Currency risks for Swiss residents


## CMD: A Short Summary

If one wants to easily replicate the main findings of this paper, they might execute the command lines below (from the root folder). Later, we'll explore each part in further detail and explore the more nuanced analysis allowed by our implementations.

In [ ]:
:: Setup
setup.bat

:: Data retrieval and processing
python src/data/fred_data_retrieval.py
python src/data/oecd_data_retrieval.py
python src/data/fred_oecd_data_concatenation.py

:: Analysis, modeling and reporting
Rscript src/models/CODE_FAMA_CURRENCY.R
python src/analysis/performances.py

:: LaTeX files compiling

## Data retrieval

We use FRED to gather ForEx and VIX monthly data through FRED's API. We additionally gather German and Italian 10y-bond yields.

We obtain short-term interest rates through an html request to the OECD database.

CAUTION: OECD data for Japan (JPY) is available only since April 2002. Thus, it's not recommended to use a start date previous to this one.

### FRED

In [1]:
import pandas as pd
import src.data.fred_data_retrieval as fred

# Define a time horizon
start_date = pd.Timestamp('2002-04-01')
end_date = pd.Timestamp('2023-10-31')

# Enter FRED API key
api_key = '41641234dac871704cd09b2b8dd163ac'

# Call to download function
fred.download(start_date=start_date,
              end_date=end_date,
              api_key=api_key)


### OECD

In [2]:
import src.data.oecd_data_retrieval as oecd

oecd.download(start_date=start_date,
              end_date=end_date)

### Merge

In [3]:
import src.data.fred_oecd_data_concatenation as merge
merge.merge()

### Load data

Now, we'll open the downloaded monthly data at a pandas Data Frame:

In [4]:
df = pd.read_csv('data/processed/monthly_data.csv')

# Standardize data handling with pandas Timestamp
df['Date'] = df['Date'].map(lambda x: pd.Timestamp(x))
df.set_index('Date', inplace=True)

# Visualize the requested data
df.head(6)[['USD', 'GBP', 'EUR', 'VIX', 'USDi', 'GBPi', 'EURi', 'IT_minus_DE']]

,USD,GBP,EUR,VIX,USDi,GBPi,EURi,IT_minus_DE
Date,,,,,,,,
2002-04-30,1.6216,2.361860,1.459764,21.91,1.87,4.17319,3.4069,0.248762
2002-05-31,1.5693,2.295101,1.465569,19.98,1.82,4.15370,3.4671,0.238273
2002-06-30,1.4920,2.274554,1.470515,25.40,1.81,4.17758,3.4640,0.238273
2002-07-31,1.4833,2.317656,1.453041,32.03,1.79,4.06054,3.4100,0.241609
2002-08-31,1.5007,2.322483,1.471586,32.64,1.73,3.98654,3.3519,0.238455
2002-09-30,1.4758,2.317006,1.457943,39.69,1.76,3.98872,3.3101,0.238455


## Analysis

### Regression results
We use R to compute the (log) forward rates and returns, and subsequently perform the linear regression in analysis.

The following command line reproduces the regression and publishes the results onto the LaTeX report, accessible at latex/paper.

### Installing IRkernel on the JupyterNotebook


First need to install and load the interactive kernel, in order to run code. In order to use the R code into the jupiterNotebook use the commande %%R.

In [5]:
!pip install rpy2 -q


In [6]:
%load_ext rpy2.ipython

- Project '~/Documents/GitHub/currency_risk_ch' loaded. [renv 1.0.3]
- The project is out-of-sync -- use `renv::status()` for details.


In [12]:
%%R
source("src/models/CODE_FAMA_CURRENCY.R")

The following package(s) will be installed:
- renv [1.0.3]
These packages will be installed into "~/Library/Caches/org.R-project.R/R/renv/library/currency_risk_ch-1d6cd68c/R-4.1/x86_64-apple-darwin17.0".

Do you want to proceed? [Y/n]: y

# Installing packages --------------------------------------------------------
- Installing renv ...                           OK [linked from cache]
- The library is already synchronized with the lockfile.
Rows: 259 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (22): USD, EUR, JPY, GBP, AUD, CAD, NZD, SEK, NOK, VIX, IT_minus_DE, TE...
date  (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 19, 2023 - 16:07:44
% Requires LaTeX packages

\hline \\[-1.8ex] 
Observations & \multicolumn{1}{c}{258} & \multicolumn{1}{c}{257} & \multicolumn{1}{c}{258} \\ 
R$^{2}$ & \multicolumn{1}{c}{0.496} & \multicolumn{1}{c}{0.392} & \multicolumn{1}{c}{0.215} \\ 
Adjusted R$^{2}$ & \multicolumn{1}{c}{0.490} & \multicolumn{1}{c}{0.385} & \multicolumn{1}{c}{0.205} \\ 
Residual Std. Error & \multicolumn{1}{c}{0.013 (df = 254)} & \multicolumn{1}{c}{0.022 (df = 253)} & \multicolumn{1}{c}{0.026 (df = 254)} \\ 
F Statistic & \multicolumn{1}{c}{83.310$^{***}$ (df = 3; 254)} & \multicolumn{1}{c}{54.480$^{***}$ (df = 3; 253)} & \multicolumn{1}{c}{23.147$^{***}$ (df = 3; 254)} \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 19, 2023 - 16:07:54
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 


Observations & \multicolumn{1}{c}{258} & \multicolumn{1}{c}{258} & \multicolumn{1}{c}{258} \\ 
R$^{2}$ & \multicolumn{1}{c}{0.463} & \multicolumn{1}{c}{0.588} & \multicolumn{1}{c}{0.651} \\ 
Adjusted R$^{2}$ & \multicolumn{1}{c}{0.454} & \multicolumn{1}{c}{0.581} & \multicolumn{1}{c}{0.646} \\ 
Residual Std. Error (df = 253) & \multicolumn{1}{c}{0.020} & \multicolumn{1}{c}{0.020} & \multicolumn{1}{c}{0.018} \\ 
F Statistic (df = 4; 253) & \multicolumn{1}{c}{54.436$^{***}$} & \multicolumn{1}{c}{90.249$^{***}$} & \multicolumn{1}{c}{118.221$^{***}$} \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 19, 2023 - 16:08:01
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{

  & & & \\ 
 Constant & 0.005 & 0.001 & 0.001 \\ 
  & (0.004) & (0.002) & (0.003) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & \multicolumn{1}{c}{237} & \multicolumn{1}{c}{237} & \multicolumn{1}{c}{237} \\ 
R$^{2}$ & \multicolumn{1}{c}{0.413} & \multicolumn{1}{c}{0.477} & \multicolumn{1}{c}{0.463} \\ 
Adjusted R$^{2}$ & \multicolumn{1}{c}{0.403} & \multicolumn{1}{c}{0.468} & \multicolumn{1}{c}{0.454} \\ 
Residual Std. Error (df = 232) & \multicolumn{1}{c}{0.024} & \multicolumn{1}{c}{0.017} & \multicolumn{1}{c}{0.021} \\ 
F Statistic (df = 4; 232) & \multicolumn{1}{c}{40.749$^{***}$} & \multicolumn{1}{c}{52.917$^{***}$} & \multicolumn{1}{c}{49.968$^{***}$} \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [ ]:
%%R
str(monthly_data)

With the str() command we can check all the variables, and source() allows use to make sure that we use the right folder. 

### Printing the regression results

The regression contains the coefficients for each parameters and the Durwin-Watson test. 


In [ ]:
%%R

print(regression_results)

The table provides an overview for the coefficient without the significance.

In [ ]:
%%R


print(dw_tests)


The Durwin watson test is great to check autocorrelation. If the results are not significance then there is no strong evidence for auticorrelation

### Robustness test
Including the spread into the regression

In [ ]:
%%R
stargazer(models_list_Robust1, align = TRUE, type = "text", out = "latex/tables/regression_table_EUR_USD_JPY_R.tex")


In [ ]:
%%R
stargazer(models_list_Robust2, align = TRUE, type = "text", out = "latex/tables/regression_table_GBP_AUD_CAD_R.tex")


In [ ]:
%%R
stargazer(models_list_Robust3, align = TRUE, type = "text", out = "latex/tables/regression_table_NZD_SEK_NOK_R.tex")


TED spread robustness check

In [14]:
%%R
stargazer(models_list_RobustTED1, align = TRUE, type = "text", out = "latex/tables/regression_table_EUR_USD_JPY_R.tex")



                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                              EUR                     USD                     JPY          
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
F_S_EUR                     -1.306                                                         
                            (1.531)                                                        
                                                                                           
F_S_USD                                              0.666                                 
                                                    (1.598)                                
                                                                               

In [16]:
%%R
stargazer(models_list_RobustTED2, align = TRUE, type = "text", out = "latex/tables/regression_table_GBP_AUD_CAD_RTED.tex")



                                    Dependent variable:      
                               ------------------------------
                                  GBP       AUD       CAD    
                                  (1)       (2)       (3)    
-------------------------------------------------------------
F_S_GBP                          0.263                       
                                (1.341)                      
                                                             
F_S_AUD                                   -0.808             
                                          (1.263)            
                                                             
F_S_CAD                                              0.029   
                                                    (2.062)  
                                                             
delta_Log_VIX                    0.008   -0.039***   -0.007  
                                (0.006)   (0.006)   (0.006)  
       

In [17]:
%%R
stargazer(models_list_RobustTED3, align = TRUE, type = "text", out = "latex/tables/regression_table_NZD_SEK_NOK_RTED.tex")



                                    Dependent variable:     
                               -----------------------------
                                  NZD       SEK       NOK   
                                  (1)       (2)       (3)   
------------------------------------------------------------
F_S_NZD                          0.260                      
                                (1.291)                     
                                                            
F_S_SEK                                   -1.154            
                                          (1.603)           
                                                            
F_S_NOK                                             -0.200  
                                                    (1.565) 
                                                            
delta_Log_VIX                  -0.038*** -0.012**  -0.024***
                                (0.007)   (0.005)   (0.006) 
                       

### FX rates in CHF

The following plot reproduces the FX rates for the specified countries list:

In [10]:
import src.analysis.performances as plot

currencies_list = ['USD', 'GBP', 'JPY']
df2 = plot.build_df(currencies_list)
returns = plot.compute_returns(df2, currencies_list=currencies_list)

AttributeError: module 'src.analysis.performances' has no attribute 'build_df'

In [ ]:
plot.plot_performances(returns)

### Excess returns on Carry FX strategy

We also analyze the profitability and risks associated with investing in the short-term rates in each of the other G-10 currencies, from the standpoint of a Swiss investor.

In [ ]:
carry_fx_xs_returns = plot.compute_carry_fx_xs_returns(df2, currencies_list=currencies_list)
plot.plot_performances(carry_fx_xs_returns)

### Performance indicators

For each of the considered strategies, one might compute historical performance measures such as the Sharpe ratio, the average annualized return and the average and maximum drawdowns through the following code. It's also possible to make drawdown plots, histograms of returns and autocorrelation bar plots.

In [ ]:
plot.plot_return_histograms(returns, currencies_list=currencies_list)
plot.plot_return_histograms(carry_fx_xs_returns, currencies_list=currencies_list)

plot.plot_autocorrelations(carry_fx_xs_returns, currencies_list=currencies_list)

### EU risk factor: DE-IT 10y Spread

The following is an indicator to be used in a robustness check of our results; it's a measure of the European risk.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df['IT_minus_DE'].plot(xlabel='Time',
                       ylabel='pp',
                       title='IT-DE 10-year bond spread')
plt.savefig('latex/figures/it_de_spread.jpg')

## LaTeX

Now, let's go through the compiling process of the LaTeX file into a pdf: